In [1]:
%reload_ext autoreload
%autoreload 2

We'll add the project to PYTHON_PATH so we can import the modules from the project.

In [2]:
import sys, os
proj = os.path.dirname(os.getcwd())
if proj not in sys.path:
    sys.path.insert(0, proj)
sys.path

['/Volumes/Projects/Evidently/record-thing',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python311.zip',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/lib-dynload',
 '',
 '/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages']

In case we use PyTorch, we'll need to have a `device` variable to handle the device where the model will be trained.
MPS support is commented out as it isn't fully supported in the DINO model that we use.

In [3]:
# Device independent code
import torch

device = "cpu" 
if torch.cuda.is_available():
    device = "cuda"
# if torch.backends.mps.is_available:
#     device = torch.device("mps")
torch.device(device)
torch.set_default_device(device=device)

os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

### Local SQLite

The sqlite database is created locally.

In [4]:
from dataset.ingestion import Ingestor
from dataset.db import init_db_sqlite

db = init_db_sqlite() # "data/processed/processed.db")
# cursor = db.cursor()
# ingestor = Ingestor(cursor, device=device)

Using cache found in /Volumes/Projects/Evidently/record-thing/dataset/.cache/torch/hub/facebookresearch_dinov2_main
/Volumes/Projects/Evidently/record-thing/dataset/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/Volumes/Projects/Evidently/record-thing/dataset/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/Volumes/Projects/Evidently/record-thing/dataset/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")


Inspecting data

### Account

In [12]:
from dataset.commons import commons
import pandas as pd

# with db.cursor() as cursor:
count = db.execute(
"""
  SELECT COUNT(*) FROM accounts
"""
        ).fetchone()

display(f"account count: {count[0]}")

rows = db.execute(
"""
  SELECT id, name, email, sms, region FROM accounts
  LIMIT 30
"""
        ).fetchall()
  
display("commons.account_id:", commons['account_id'])
display(pd.DataFrame(rows, columns=["id", "name", "email", "sms", "region"]))

owners = db.execute("SELECT * FROM owners LIMIT 1")
print("Owners")
display(pd.DataFrame(owners.fetchall(), columns=[desc[0] for desc in owners.description]))

'account count: 1'

'commons.account_id:'

b'2pvhs6P5YBKq4NAYas1NED6PehV'

,id,name,email,sms,region
0,b'2pvhs6P5YBKq4NAYas1NED6PehV',Joe Schmoe,None,None,None


Owners


,account_id,created_at


### Clip Assets

In [14]:
import pandas as pd

# with connection.cursor() as cursor:
count = db.execute(
"""
  SELECT COUNT(*) FROM clip_assets
"""
        ).fetchone()

display(f"clip_assets count: {count[0]}")

rows = db.execute(
"""
  SELECT clip_assets.id, clip_assets.name FROM clip_assets
  INNER JOIN dino_embedding ON dino_embedding.asset_id = clip_assets.id
  LIMIT 30
"""
        ).fetchall()


display(pd.DataFrame(rows, columns=["embedding", "distance", "name"]))

[autoreload of dataset.db.assets failed: Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 500, in superreload
    update_generic(old_obj, new_obj)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 397, in update_generic
    update(a, b)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 353, in update_class
    setattr(old, key, getattr(new, key))
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sqlmodel/main.py", line 484, in __setattr__
    DeclarativeMeta.__setattr__(cls, name, value)
  File "/opt/homebrew/Caskroom/miniconda/base/lib/python3.11/site-packages/sq

'clip_assets count: 0'

,embedding,distance,name


### Products

In [13]:
import pandas as pd

# with connection.cursor() as cursor:
count = db.execute(
"""
  SELECT COUNT(*) FROM products
"""
        ).fetchone()

display(f"clip_assets count: {count[0]}")

rows = db.execute(
    """
      SELECT id, name, upc, asin, elid, brand, model, color, tags, category, title, description FROM products
      LIMIT 30
    """
    )


# display(pd.DataFrame(rows, columns=["id", "name", "upc","asin","elid","brand","model","color","tags","category","title","description"]))
display(pd.DataFrame(rows.fetchall(), columns=[desc[0] for desc in rows.description]))


'clip_assets count: 0'

,id,name,upc,asin,elid,brand,model,color,tags,category,title,description
